# Limestone Data Challenge
### Contributers: Pratik Sahoo, Satyankar Chandra

Hi and welcome to our submission!
We will try our best to keep an engaging commentary running with the code. The code represents our final solution, but with our text blocks we will try to give you some insight into how we reached this.

Let's begin by importing necessary libraries.

In [3]:
# --------------- IMPORTS ----------------

# Data Handling Tools
import csv
import numpy as np
import math

# For clustering of stocks into sectors
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

# For making neural net models of indices
import torch
from torch import nn
from torch import optim

# For data visualisation
import matplotlib.pyplot as plt

# For random initialization of the trading strategy vector
import random

# For making waiting fun :)
from tqdm import tqdm

The next two blocks help extract data from the .csv files provided, and then extract the returns from them, converting them into basis points.

In [4]:
stock_prices = []

with open("data_challenge_stock_prices.csv", 'r') as csvfile:
    csvreader = csv.reader(csvfile)
     
    next(csvreader)
 
    for row in csvreader:
        stock_prices.append(row)
 

index_prices = []

with open("data_challenge_index_prices.csv", 'r') as csvfile:
    csvreader = csv.reader(csvfile)
     
    next(csvreader)
 
    for row in csvreader:
        index_prices.append(row)

stock_prices = np.array(stock_prices, dtype=float)
index_prices = np.array(index_prices, dtype=float)


In [5]:
stock_returns = [] # Both contain values in Basis Points
index_returns = []

for i in range(199999):
    stock_returns.append(10000 * (stock_prices[i+1] - stock_prices[i]) / stock_prices[i])
    index_returns.append(10000 * (index_prices[i+1] - index_prices[i]) / index_prices[i])

stock_returns = np.array(stock_returns, dtype=float)
index_returns = np.array(index_returns, dtype=float)

Since the first two problem statements require us to find the partitioning of stocks into sectors with no other data than the returns, our first intuition was to find the correlation between them because:
- we lacked any other information linking different stocks
- we believed that different stocks of the same sector would exhibit high correlation due to similar market forces.

In [6]:
def correlation(X, Y): # assumes X and Y same length

    n = 0
    d1 = 0
    d2 = 0
    xm = 0
    ym = 0
    n = X.__len__()

    for i in range(n):
        xm += X[i]
        ym += Y[i]
    
    xm /= n
    ym /= n

    for i in range(n):
        n += (X[i] - xm)*(Y[i] - ym)
        d1 += (X[i] - xm) ** 2
        d2 += (Y[i] - ym) ** 2

    return (n*1.0)/(math.sqrt(d1*d2))

In [7]:
correl = np.array([[0.1 for _ in range(100)] for _ in range(100)])

for i in tqdm(range(100)):
    for j in range(100):
        if i==j:
            correl[i][j] = 1
            continue
        if i>j:
            correl[i][j] = correl[j][i]
            continue
        X = stock_returns[:, i]
        Y = stock_returns[:, j]
        correl[i][j] = np.corrcoef(X, Y)[0,1]


100%|██████████| 100/100 [00:10<00:00,  9.19it/s]


In [8]:
file = open("file_corr.txt", "w+")
 
# Saving the array in a text file
for i in range(100):
    content = str(correl[i])
    file.write(content)

file.close()


In [9]:
dissimilarity = 1 - correl
hierarchy = linkage(squareform(dissimilarity), method='average')
labels = fcluster(hierarchy, 0.88, criterion='distance')
for j in range(len(labels)):
    labels[j] -= 1

In [10]:
# for i in range(100):
#     print(i, labels[i])

clusters = dict()
for i in range(100):
    if labels[i] not in clusters:
        clusters[labels[i]] = [i]
    else:
        clusters[labels[i]].append(i)


for i in range(0, len(clusters)):
    print(i, clusters[i])

0 [5, 13, 15, 24, 31, 34, 35, 37, 48, 56, 61, 62, 64, 65, 68, 70, 76, 83, 85, 86, 89, 93, 94, 95, 97]
1 [2, 3, 6, 7, 9, 16, 17, 19, 23, 28, 32, 33, 42, 47, 51, 53, 58, 60, 63, 66, 72, 81, 87, 91, 98]
2 [8, 12, 27, 29, 30, 36, 40, 41, 44, 46, 50, 52, 55, 57, 59, 69, 71, 73, 74, 77, 80, 84, 92, 96, 99]
3 [0, 1, 4, 10, 11, 14, 18, 20, 21, 22, 25, 26, 38, 39, 43, 45, 49, 54, 67, 75, 78, 79, 82, 88, 90]


In [9]:
# print(1, correl[0][1])
# print(2,correl[1][14])
# print(3,correl[0][14])
# print(4,correl[1][26])
# print(5,correl[14][20])
# print(6,correl[88][90])
# print(7,correl[75][38])
# print(8,correl[14][39])
# print(9,correl[26][43])
# print(10,correl[39][45])
# print(11,correl[1][49])
# print(12,correl[38][75])

In [10]:
def cross_rel(x, y):
    re = []
    for i in range(100):
        for j in range(100):
            if labels[i] == x and labels[j] == y:
                re.append(correl[i][j])

    return re




In [11]:
for p in range(4):
    a = cross_rel(3, p)
    print(p, round(100*np.mean(a), 3))


0 3.608
1 3.79
2 4.06
3 19.534


In [12]:
stock_group_correl = [[0.0 for _ in range(15)] for _ in range(100)]
for i in range(100):
    for j in range(15):
        X = stock_returns[:, i]
        Y = index_returns[:, j]
        stock_group_correl[i][j] = np.corrcoef(X,Y)[0,1]

In [13]:
def group_index_analyze(grp, ind):
    for stck in clusters[grp]:
        print(stck, stock_group_correl[stck][ind])

In [14]:
group_index_analyze(0,5)

5 -0.043444624028860825
13 -0.054021430920410674
15 -0.11968848181583566
24 -0.055484313088256194
31 -0.04956152382931762
34 -0.04170376823782463
35 -0.054467331363896546
37 -0.05227058212103044
48 -0.05637425827865387
56 -0.058204794919400775
61 -0.06425740896053413
62 -0.06291187068245913
64 -0.06253907102175497
65 -0.0643838233217589
68 -0.06990123288624457
70 -0.06496109876331742
76 -0.06806300599969749
83 -0.06934472949092621
85 -0.08595294220833403
86 -0.18991076019572092
89 -0.08645366951399902
93 -0.08655172741848911
94 -0.14035460483535137
95 -0.09149518993715464
97 -0.10238044631424775


In [16]:
data = [[[0 for _ in range(25)] for _ in range(199999)] for _ in range(4)]

p=0
for i in clusters[0]:
    
    for j in range(199999):
        data[0][j][p] = stock_returns[j][i]
    
    p +=1

p=0
for i in clusters[1]:
    
    for j in range(199999):
        data[1][j][p] = stock_returns[j][i]
    
    p +=1
p=0
for i in clusters[2]:
    
    for j in range(199999):
        data[2][j][p] = stock_returns[j][i]
    
    p +=1

p=0

for i in clusters[3]:
    for j in range(199999):
        data[3][j][p] = stock_returns[j][i]
    
    p +=1


# data4 = torch.FloatTensor(data3)

# batches = 200

# model = nn.Sequential(nn.Linear(25,64),nn.ReLU(),nn.Linear(64,64),nn.ReLU(),nn.Linear(64,1))
# facts = nn.L1Loss()
# optimizer = optim.Adam(model.parameters(), lr = 0.0001)

# for e in tqdm(range(500)):
#     for k in range(batches):
#         optimizer.zero_grad()
#         output = model(data4[int(k*199999/batches):int((k+1)*199999/batches), :])
#         loss = facts(output, torch.unsqueeze(torch.FloatTensor(index_returns[int(k*199999/batches):int((k+1)*199999/batches),0]), 1))
#         if ((k == 0 and e==0) or k == batches-1) and e%100 == 0:
#             print(loss.item())
#         loss.backward()
#         optimizer.step()
        
        

In [36]:
def make_model_for_index(index, data, epochs=500): # data contains a 199999x25 array containing returns of a sector

    batches = 200
    model = nn.Sequential(nn.Linear(25,64),nn.ReLU(),nn.Linear(64,256),nn.ReLU(),nn.Linear(256,64),nn.ReLU(),nn.Linear(64,1))
    facts = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr = 0.001)

    data4 = torch.FloatTensor(data)



    for e in tqdm(range(epochs)):
        for k in range(batches):
            optimizer.zero_grad()
            output = model(data4[int(k*data.__len__()/batches):int((k+1)*data.__len__()/batches), :])
            loss = facts(output, torch.unsqueeze(torch.FloatTensor(index_returns[int(k*data.__len__()/batches):int((k+1)*data.__len__()/batches),index]), 1))

            l2_lambda = 0.0008
            l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
 
            loss = loss + l2_lambda * l2_norm
            loss.backward()
            optimizer.step()
    
    return model

In [18]:
# model0_3 = make_model_for_index(9, data[2], epochs=800)

In [19]:
# output = np.array(torch.squeeze(model0_3(torch.FloatTensor(data[2])).detach()))
# target = np.array(index_returns[:, 9])
# print(np.mean(np.abs(output-target)))

In [40]:
index_to_group = [3,0,3,1,2,0,1,3,1,2,0,3,2,1,2] # TODO: Fill
index_models = [] # will contain models trained on first 170k samples, will verify on rest 30k
pred_correl = []

for i in range(15):
    index_models.append(make_model_for_index(i, data[index_to_group[i]][:170000][:], epochs=30))
    predictions = np.array(torch.squeeze(index_models[-1](torch.FloatTensor(data[index_to_group[i]][170000:][:])).detach()))
    pred_correl.append(np.corrcoef(predictions,[index_returns[p][i] for p in range(170000, 199999)])[0,1])
    print(i, 100*pred_correl[-1])

100%|██████████| 30/30 [00:09<00:00,  3.08it/s]


0 44.61740303456987


100%|██████████| 30/30 [00:10<00:00,  2.85it/s]


1 40.304720120482266


100%|██████████| 30/30 [00:10<00:00,  2.86it/s]


...

12 43.59207690891128


100%|██████████| 30/30 [00:10<00:00,  2.82it/s]


13 44.14143524136973


100%|██████████| 30/30 [00:11<00:00,  2.69it/s]

14 53.644559431567885


In [41]:
mu = []
for i in range(15):
    mu.append(np.array(torch.squeeze(index_models[-1](torch.FloatTensor(data[index_to_group[i]][:][:])).detach())))

mu = np.array(mu)

covar = np.cov(mu)
print(covar.shape)

(15, 15)


In [51]:
def normalize_alpha(A): # A is the 1-d array of allocations, we will use this function to satisfy 2nd and 3rd bullet points of 5th problem statement
    A = np.array(A)
    mn = A.mean()
    A = A - np.array([mn for _ in range(len(A))])
    mx = max(abs(A))
    for i in range(len(A)):
        A[i] /= mx

    return A

In [70]:
def Alpha(mu_t, covv): # takes Mu_t and covariance matrix and returns A (allocation)
    
    """-----------MAKE STRATEGY HERE---------------
    and preferably explain it"""
    cst = 100
    cst_rt = 10
    old_cst = 200
    gamma = 0.13
    lr = 0.002

    A = np.array([random.random() for _ in range(15)])

    while( abs(cst - old_cst) > 0.001 ):
        A = normalize_alpha(A)
        old_cst = cst
        cst = np.dot(A,mu_t) - gamma * np.sum([[A[i]*A[j]*covv[i][j] for i in range(15)] for j in range(15)])
        
        grad = np.array([mu_t[g] - gamma * 2 * np.sum([covv[g][h] * A[h] for h in range(15)]) for g in range(15)])

        A = A + lr * grad




    """================ REMEMBER TO NORMALIZE===================="""

    A = normalize_alpha(A)

    return A


In [71]:
def test_Alpha(Alph):
    day_ret = []

    for t in tqdm(range(191550,191599)):

        A = Alpha([mu[p][t] for p in range(15)], covar)
        actual_returns = [index_returns[t][p] for p in range(15)]
        day_ret.append(np.dot(A,actual_returns))

    day_ret = np.array(day_ret)

    print(f"Mean daily returns: {day_ret.mean()}")
    print(f"Standard deviation: {day_ret.std()}")
    print(f"Sharpe ratio: {day_ret.mean()/day_ret.std()}") 
    # Since long and short positions net out, the risk-free return rate is 0 (not buying nor selling).
    # Hence, in Sharpe formula, Ra - Rb = Ra

In [72]:
test_Alpha(Alpha)

100%|██████████| 49/49 [00:07<00:00,  6.37it/s]

Mean daily returns: 1.920291763029162
Standard deviation: 4.401499645481538
Sharpe ratio: 0.43628124905121424
